<a href="https://colab.research.google.com/github/shubhamyadav1107/Personal_portfolio_website/blob/main/RAG_for_QA_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install faiss-cpu
!pip install transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 39.8 MB/s eta 0:00:00


In [2]:
import faiss
import numpy as np
from transformers import AutoTokenizer, AutoModel
import torch


In [3]:
# Loading pre-trained tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

# Function to generate embeddings from text
def generate_embeddings(text):
    tokens = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        embeddings = model(**tokens).last_hidden_state.mean(dim=1)
    return embeddings[0].numpy()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

In [4]:
# Sample text data (text given in the form of a list and each element represents different section of the sample data)
text_data = [
    "Company Overview: TechFusion Inc. is a leading provider of innovative technology solutions, specializing in cloud computing, cybersecurity, and data analytics. Founded in 2010, the company has grown to serve over 500 clients worldwide, offering tailored solutions to help businesses enhance their operations through digital transformation",
    "Services Offered: 1. Cloud Computing Solutions: TechFusion provides cloud architecture design, migration services, and ongoing cloud management for businesses looking to scale. 2. Cybersecurity: Our team ensures that your company’s data is protected with state-of-the-art security protocols, risk assessments, and incident response services. 3. Data Analytics: We offer custom analytics solutions that help businesses derive actionable insights from their data, using AI and machine learning models",
    "Client Testimonials: TechFusion helped us transition to the cloud smoothly, saving us time and operational costs. - Sarah Johnson, CEO of DigitalCorp Their cybersecurity team is second to none. We feel much safer knowing TechFusion is protecting our systems. - Tom Parker, CTO of FinTech Global",
    "Case Studies: 1. FinTech Global: TechFusion helped this financial services company migrate to a secure cloud infrastructure, reducing operational costs by 30% and increasing system uptime by 25%.  2. RetailCo: Our data analytics solution enabled RetailCo to identify customer buying patterns, resulting in a 15% increase in sales during the last quarter",
    "FAQs: Q1: What industries does TechFusion specialize in? A1: TechFusion serves a variety of industries, including finance, retail, healthcare, and education. Q2: How does TechFusion ensure data security?  A2: We implement multi-layer security protocols, regular audits, and real-time monitoring to ensure data is secure at all times. Q3: What is the process for engaging with TechFusion?  A3: Clients typically start with a consultation to assess their needs, followed by a proposal and a structured project plan.",
]

# Generating embeddings for each text segment
embeddings = [generate_embeddings(text) for text in text_data]


In [5]:
#initializing FAISS Index Next, we’ll initialize FAISS and add the generated embeddings to it. For this project, we’ll use the L2 distance for similarity search.

In [6]:
# Defining the dimension of the embeddings, 768 for this transformer model
embedding_dim = 384

# Initializing the FAISS index (L2 distance is used here)
index = faiss.IndexFlatL2(embedding_dim)

# Converting the embeddings to a numpy array of type float32
embeddings_array = np.array(embeddings).astype('float32')

# Adding the embeddings to the FAISS index
index.add(embeddings_array)


In [7]:
#Query FAISS
# After we’ve stored the embeddings in FAISS, we can perform similarity searches. Let’s take a sample query, generate its embedding, and retrieve the most similar text segments.


# Example query to test retrieval
query = "What services does TechFusion offer?"

# Generating an embedding for the query
query_embedding = generate_embeddings(query).reshape(1, -1)

# Searching the FAISS index for the top 3 most similar embeddings
D, I = index.search(query_embedding, 3)  # D = distances, I = indices of closest matches

# Print the top results
print(f"Top results (indices): {I[0]}")

for idx in I[0]:
    print(f"Retrieved text: {text_data[idx]}")


Top results (indices): [0 2 4]
Retrieved text: Company Overview: TechFusion Inc. is a leading provider of innovative technology solutions, specializing in cloud computing, cybersecurity, and data analytics. Founded in 2010, the company has grown to serve over 500 clients worldwide, offering tailored solutions to help businesses enhance their operations through digital transformation
Retrieved text: Client Testimonials: TechFusion helped us transition to the cloud smoothly, saving us time and operational costs. - Sarah Johnson, CEO of DigitalCorp Their cybersecurity team is second to none. We feel much safer knowing TechFusion is protecting our systems. - Tom Parker, CTO of FinTech Global
Retrieved text: FAQs: Q1: What industries does TechFusion specialize in? A1: TechFusion serves a variety of industries, including finance, retail, healthcare, and education. Q2: How does TechFusion ensure data security?  A2: We implement multi-layer security protocols, regular audits, and real-time mon

In [8]:
#FAISS testing successfull and we can see that:
# 1. FAISS will retrieve the top 3 most similar text sections based on the query.
# 2. The retrieved results will be printed out as shown above, showing how well the embeddings match the query.

In [9]:
!pip install cohere


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.4/222.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.1 MB/s eta 0:00:00


In [10]:
import cohere

# Initializing cohere client with our API key
cohere_api_key = 'leOdbUJerRwmQejRSlDqIIECdwMJQpSg5NNbgb04'
co = cohere.Client(cohere_api_key)


In [11]:
# Example query to test retrieval
query = "What services does TechFusion provide?"

# Generate an embedding for the query
query_embedding = generate_embeddings(query).reshape(1, -1)

# Search the FAISS index for the top 3 most similar embeddings
D, I = index.search(query_embedding, 3)

# Retrieve the top results from your dataset
retrieved_texts = [text_data[idx] for idx in I[0]]
print("Retrieved texts: ", retrieved_texts)


Retrieved texts:  ['Company Overview: TechFusion Inc. is a leading provider of innovative technology solutions, specializing in cloud computing, cybersecurity, and data analytics. Founded in 2010, the company has grown to serve over 500 clients worldwide, offering tailored solutions to help businesses enhance their operations through digital transformation', 'Client Testimonials: TechFusion helped us transition to the cloud smoothly, saving us time and operational costs. - Sarah Johnson, CEO of DigitalCorp Their cybersecurity team is second to none. We feel much safer knowing TechFusion is protecting our systems. - Tom Parker, CTO of FinTech Global', 'FAQs: Q1: What industries does TechFusion specialize in? A1: TechFusion serves a variety of industries, including finance, retail, healthcare, and education. Q2: How does TechFusion ensure data security?  A2: We implement multi-layer security protocols, regular audits, and real-time monitoring to ensure data is secure at all times. Q3: Wh

In [13]:
# Combining the retrieved text into one input for the generative model
retrieved_text = "\n".join(retrieved_texts)

# Prompt for the generative model
prompt = f"Based on the following information, answer the question: '{query}'\n\nInformation:\n{retrieved_text}\n\nAnswer:"

# Call the Cohere API to generate a response
response = co.generate(
    model='command-xlarge-nightly',
    prompt=prompt,
    max_tokens=100,  # length of the generated answer
    temperature=0.7,  #  You can adjust the creativity of the answer (0.7 is moderate)
)

# Extract the generated answer
answer = response.generations[0].text.strip()
print(f"Generated Answer: {answer}")


Generated Answer: TechFusion Inc. provides a range of technology services, primarily focusing on cloud computing, cybersecurity, and data analytics. They assist businesses in their digital transformation journey by offering tailored solutions that include:
1. Cloud Computing: Helping companies migrate their operations to the cloud, making it more efficient, cost-effective, and accessible.
2. Cybersecurity: Employing advanced security measures, such as multi-layer protocols, audits, and real-time monitoring, to protect client systems and data from potential threats


In [15]:
# Testing with Example Queries
# We can now test the entire pipeline by running queries through FAISS and generating answers using Cohere.

# 1.Input a query.
# 2.Retrieve the top matching text from FAISS.
# 3.Feed that information to Cohere to generate a natural language response.

# Queries like:

#“What services does TechFusion provide?”
#“What is the process for engaging with TechFusion?”
#“How does TechFusion ensure data security"

In [16]:
# Example query to test retrieval
query = "What is the process of engaging with TechFusion?"

# Generate an embedding for the query
query_embedding = generate_embeddings(query).reshape(1, -1)

# Search the FAISS index for the top 3 most similar embeddings
D, I = index.search(query_embedding, 3)

# Retrieve the top results from your dataset
retrieved_texts = [text_data[idx] for idx in I[0]]
print("Retrieved texts: ", retrieved_texts)

Retrieved texts:  ['Company Overview: TechFusion Inc. is a leading provider of innovative technology solutions, specializing in cloud computing, cybersecurity, and data analytics. Founded in 2010, the company has grown to serve over 500 clients worldwide, offering tailored solutions to help businesses enhance their operations through digital transformation', 'FAQs: Q1: What industries does TechFusion specialize in? A1: TechFusion serves a variety of industries, including finance, retail, healthcare, and education. Q2: How does TechFusion ensure data security?  A2: We implement multi-layer security protocols, regular audits, and real-time monitoring to ensure data is secure at all times. Q3: What is the process for engaging with TechFusion?  A3: Clients typically start with a consultation to assess their needs, followed by a proposal and a structured project plan.', 'Client Testimonials: TechFusion helped us transition to the cloud smoothly, saving us time and operational costs. - Sarah

In [17]:
# Combine the retrieved text into one input for the generative model
retrieved_text = "\n".join(retrieved_texts)

# Prompt for the generative model
prompt = f"Based on the following information, answer the question: '{query}'\n\nInformation:\n{retrieved_text}\n\nAnswer:"

# Call the Cohere API to generate a response
response = co.generate(
    model='command-xlarge-nightly',
    prompt=prompt,
    max_tokens=100,  # length of the generated answer
    temperature=0.7,  #  You can adjust the creativity of the answer (0.7 is moderate)
)

# Extract the generated answer
answer = response.generations[0].text.strip()
print(f"Generated Answer: {answer}")

Generated Answer: The process of engaging with TechFusion typically begins with a consultation where potential clients meet with TechFusion's team to assess their specific needs and requirements. During this initial consultation, TechFusion gathers information about the client's business, their current challenges, and the goals they want to achieve through digital transformation. 

After the consultation, TechFusion prepares a detailed proposal outlining the recommended solutions, services, and a timeline for the project. This proposal is tailored to the client's unique needs, ensuring a customized approach


In [18]:
#Now creating a simple web based user interface for a lay man using Gradio so he/she can directly write down the querry and get the generated cohorrent answer from the text which is already saved in the form of embeddings in the vector DB

In [20]:
# We already have the logic for retrieving data from FAISS and generating an answer using Cohere. We’ll encapsulate this logic in a function that Gradio can use.

In [22]:
# Creating a Gradio Interface: We’ll use Gradio’s Textbox component for query input and display the generated answer in a Textbox as well.

In [33]:
!pip install gradio --upgrade

In [36]:


import gradio as gr

# Function to handle query input, retrieval, and generation

def answer_question(query):
    # Step 1: Generate embedding for the query
    query_embedding = generate_embeddings(query).reshape(1, -1)

    # Step 2: Retrieve the most similar texts from FAISS
    D, I = index.search(query_embedding, 3)
    retrieved_texts = [text_data[idx] for idx in I[0]]
    retrieved_text = "\n".join(retrieved_texts)

    # Step 3: Create a prompt for Cohere
    prompt = f"Based on the following information, answer the question: '{query}'\n\nInformation:\n{retrieved_text}\n\nAnswer:"

    # Step 4: Generate the answer using Cohere
    response = co.generate(
        model='command-xlarge-nightly',
        prompt=prompt,
        max_tokens=100,
        temperature=0.7
    )

    # Step 5: Extract and return the generated answer
    answer = response.generations[0].text.strip()
    return answer


# Create the Gradio interface
interface = gr.Interface(
    fn=answer_question,  # The function to handle the query
    inputs=gr.Textbox(lines=2, placeholder="Enter your question here..."),  # Input field for the query
    outputs="text",  # Output area for the generated answer
    title="TechFusion QA Bot",  # Title of the UI
    description="Ask any question related to TechFusion Inc. and get an answer generated from the provided information."
)

# Launch the interface
interface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://b5f552a8f576433774.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
